# Der GfM-Newsletter als Forschungsdatum
## Präsentation zum Vortrag auf der Jahrestagung 2022 der Gesellschaft für Medienwissenschaft (GfM): «Arbeit»
Martin-Luther-Universität Halle-Wittenberg, 28. September – 01. Oktober 2022  
Dr. Dietmar Kammerer, Philipps-Universität Marburg

# ABSTRACT

Der GfM-Newsletter ist ein Forschungsdatum. Das Archiv der Newsletter 2015-2021 enthält rund 1000 Stellenanzeigen. Diese werden nach Datum, Standort, Stellenbeschreibung, Besoldungsgruppe ausgewertet. Auf diese Weise wird die Entwicklung von Stellen in der Medienwissenschaft (im deutschsprachigen Raum) untersuchbar.  
Die Studie ist dabei explorativ und dient der Veranschaulichung der aktuellen Möglichkeiten, aber auch der Grenzen von datenbasierter Forschung im Fach. Alle verwendeten Daten, Tools und Visualisierungen werden über GitHub bereit gestellt. Der Vortrag lädt ein zur Diskussion, welche Einrichtungen (Infrastrukturen, Praktiken) helfen könnten, bestehende Defizite und Desiderate datenbasierter Forschung abzubauen.

Dieses Notebook nimmt Newsletter der Gesellschaft für Medienwissenschaft (GfM) und …
- erstellt ein Archiv der Mails, umbenannt im Format `YYYY-MM-DD.eml`
- sichert PDF-Anhänge (`YYYY-MM-DD_Anhang.pdf`)
- erstellt Tabelle "meldungen.csv" mit sämtlichen Newsletter-Einträgen
- erstellt Tabelle "stellenanzeigen.csv" mit den Volltexten der Stellenanzeigens
- bietet Analysetools an

## Vorbemerkungen
Mein Vortrag heute verfolgt zwei Ziele: **Erstens** will ich vorführen, was es heißt, den Newsletter der GfM als schwach strukturiertes Forschungsdatum zu begreifen, das heißt: Ich will die Frage stellen, wie leicht – oder wie mühsam – es ist, ihn mit einfach verfügbaren Tools und Techniken unter bestimmten Gesichspunkten automatisiert auszuwerten und dabei Forschunsgdaten zu produzieren. 
Es geht mir also weniger um das Ergebnis meiner Auswertung - obwohl ich eine Reihe von Ergebnissen in Form von Diagrammen mitgebracht habe - als um die Methode, den Weg dorthin. Genauer: Um die Hürden, Hindernisse und Stolperfallen auf diesem Weg, aber auch um die Hilfsmittel und Techniken, die einem zur Verfügung stehen.
»Arbeit«, das Thema dieser Jahrestagung, taucht also zweimal, in doppelter gestalt auf: Einmal in Form der **Stellenanzeigen**, die über den Newsletter in Community der Mitglieder der Gesellschaft für Medienwissenschaftler verbreitet werden. Diese Anzeigen versprechen Arbeit in Form eines vertraglich gesicherten Anstellungsverhältnisses an einer wissenschaftlichen Institution – und wir wissen, wie ungesichert, prekär und vorübergehend solche vertraglich gesicherten Verhältnisse sind. Zum anderen geht es um wissenschaftliches Arbeiten, um wissenschaftliche Forschungspraxis, in diesem Fall: um Forschungsdatenpraxis.
Welche Tools stehen dafür zur Verfügung? Welche Möglichekten haben wir im Umgang mit Forschungsdaten, welche Schwierigkeiten bestehen dabei? Insofern ist dieser Vortrag explorativ zu verstehen: Er erkundet ein für unser Fach immer noch relativ neues methodisches Terrain, an einem konkreten Beispiel.

Damit ist auch schon das **zweite Ziel** benannt: Alle Daten, alle Techniken, alle Ergebnisse des Vortrages können eingesehen und nachgenutzt werden - das sind die **FAIR-Prinzipien des Forschungsdatenmanagements**. Auf diese Weise können die Ergebnisse meines kleinen Forschungsprojekts nicht nur nachvollzogen, sondern auch repliziert oder reproduziert werden. Ich werde später vorstellen, welche Ressourcen und Dokumente das genau umfasst und wo diese zu finden sind.

Eine letzte Vorbemerkung, bevor wir zu den Inhalten kommen: Ich präsentiere hier die Ergebnisse eines **kleinen Forschungsprojekts**. Es ist ein Nebenbeiprojekt, eine Fingerübung, kein Schwerpunkt meiner aktuellen Tätigkeit oder Stellenbeschreibung. Mir ist bewußt, dass das dahinter stehende Thema  - die Entwicklung der Stellensituation in Deutschland in der Medienwissenschaft - selbstverständlich noch viel mehr Aufmerksamkeit und Genauigkeit und detaillierte Analysen verdient hätte, was ich aber - im Rahmen meines durch Drittmittel nur temporär gesichert/ungesicherten Arbeitsverhältnisses - nicht leisten konnte. Was ich heute vorstelle, ist in den Abendstunden oder an Wochenenden entstanden.
Andererseits ist ein Ziel des Projekts ja gerade darin, Daten und Ressourcen bereit zu stellen, damit Dritte diese nachnutzen können, um weitere und eigene Fragen zu stellen, eigene Analysen durchzuführen und zu eigenen Ergebnissen zu kommen.


## Datengrundlage
- **GfM-Newsletter** im Zeitraum **vom 30.01.2015 bis 29.04.2022**
- **318 Newsletter**

## Strukturierte Daten im Newsletter
![Newsletter](screenshot_newsletter.png)

# VORBEREITUNG

## Importe

In [ ]:
# Diese Zelle importiert die notwendigen Module
import email
from email import policy
from email.parser import BytesParser
from io import BytesIO
import os
import pandas
import matplotlib.pyplot as plt
from pathlib import Path
from pdfminer.high_level import extract_text
import re
import shutil

Tabellen im Notebook werden auf ganzer Seite und mit Scrollbalken angezeigt

In [ ]:
# Tabellen im Notebook werden auf ganzer Seite und mit Scrollbalken angezeigt
pandas.set_option('display.max_row', None)
pandas.set_option('display.max_colwidth', None)

## Definition der Klassen ‚Mail‘ und ‚Anhang‘

In [ ]:
class Mail:
    
    '''definiert Klasse Mail'''
    
    def __init__(self, mailfile):
        
        self.path = Path(mailfile)
        self.msg = email.message_from_binary_file(open(mailfile, 'rb'), policy=policy.default)
        self.date = self.msg['Date'].datetime.isoformat().split('T')[0]
        
        self.volltext = self.get_volltext()
        self.meldungen = self.get_meldungen()
        
    def __repr__(self):
        return f'{self.volltext}'
    
    def get_volltext(self):
        return self.msg.get_body(preferencelist=('plain')).get_content()        

    def get_meldungen(self):
        records = [{'Datum': self.date, 'Meldung': re.sub('\(\d+\)','', line).strip()} \
                   for line in re.findall('\(\d+\)\s*\n?.*', self.volltext)]
        df = pandas.DataFrame.from_dict(records)
        df.Datum = pandas.to_datetime(df.Datum)
        return df
    

In [ ]:
class Anhang:

    '''definiert Klasse Anhang'''

    def __init__(self, mailfile):
        
        self.path = Path(mailfile)
        self.msg = email.message_from_binary_file(open(mailfile, 'rb'), policy=policy.default)
        self.date = self.msg['Date'].datetime.isoformat().split('T')[0]
        self.volltext = self.read_attachment()
        self.anzeigen = self.get_stellenanzeigen()
    
    def __repr__(self):
        return self.volltext[:150]
    
    def read_attachment(self):
        for file in self.msg.iter_attachments():
            if "Newsletter" in file.get_filename():
                try:
                    volltext = extract_text(BytesIO(file.get_payload(decode=True))).strip()
                    return volltext

                except:
                    print("Kann Anhang nicht auslesen.")
                    return None

    def get_stellenanzeigen(self):
        anzeigen = re.split('\(\d+\)\s+Stellenauss?chreibung:?\s+', self.volltext)

        if len(anzeigen) > 1:
            records = [{'Datum': self.date, 'Stellenanzeige': anzeige.strip()} for anzeige in anzeigen[1:]]
            df = pandas.DataFrame.from_dict(records)
            df.Datum = pandas.to_datetime(df.Datum)
            return df

        else:
            return None
        
    def save_as_pdf(self, outfolder='.'):    # save attachment as PDF
        for file in self.msg.iter_attachments():
            if "Newsletter" in file.get_filename():
                fo = Path(outfolder, f'{self.date}_anhang.pdf')
                try:
                    fo.write_bytes(file.get_payload(decode=True))
                except:
                    print("Kann Anhang nicht speichern")
                    return None

# I. DATENARCHIV ERSTELLEN

## Umgebungsvariablen festlegen

In [ ]:
# In dieser Zelle werden u.a. die Variablen für In- und Output festgelegt
infolder = 'newsletter'
outfolder = 'out'    

infolder = Path(infolder)   # Materialordner
outfolder = Path(outfolder) # hier wird der Output gespeichert
Path.mkdir(outfolder, exist_ok = True) # bestehender Ordner wird nicht überschrieben

count = len(os.listdir(infolder)) # Anzahl Dateien in Archiv

meldungen = pandas.DataFrame() # legt leeren Dataframe an
stellenanzeigen_volltext = pandas.DataFrame() # legt leeren Dataframe an

## DataFrame "Meldungen" wird erstellt

In [ ]:
# Diese Zelle extrahiert alle Meldungen aus den Mails
# Ergebnis wird unter meldungen.csv gespeichert

for file in infolder.glob('*.eml'):
    mail = Mail(file)
    meldungen = pandas.concat([meldungen, mail.meldungen], ignore_index=True)

meldungen = meldungen.sort_values(by='Datum', ignore_index=True) # sortiert Meldungen nach Datum

fo = Path(outfolder, 'meldungen.csv')
meldungen.to_csv(fo, index=False, encoding='utf8')

Diese Zelle sichert alle Mails als `YYYY-MM-DD.eml` und alle Anhänge als `YYYY-MM-DD_anhang.pdf`.  
Die Volltexte der Stellenanzeigen wird als `stellenanzeigen_volltext.csv` gesichert.

In [ ]:
# Diese Zelle sichert alle Mails als `YYYY-MM-DD.eml`
# und alle Anhänge als `YYYY-MM-DD_anhang.pdf`
# Die Volltexte der Stellenanzeigen wird als 'stellenanzeigen_volltext.csv' gesichert.

for n, file in enumerate(infolder.glob('*.eml')):
    
    mail = Mail(file)
    anhang = Anhang(file)
    
    print(f'Verarbeite Newsletter vom {mail.date}. Mail {n+1} von {count-1}')
    
    fo = Path(outfolder, f'{mail.date}.eml')
    shutil.copy(file, fo) # sichere Mail als Kopie, umbenannt in YYYY-MM-DD.eml
    
    anhang.save_to_pdf(outfolder)
    
    meldungen = pandas.concat([meldungen, mail.meldungen], ignore_index=True)
    
    stellenanzeigen_volltext = pandas.concat([stellenanzeigen_volltext, anhang.anzeigen], ignore_index=True)
    
# Volltexte aller Stelenanzeigen sichern
fo = Path(outfolder, 'stellenanzeigen_volltext.csv')
stellenanzeigen_volltext.to_csv(fo, index=False, encoding='utf8')


# II. DATEN AUFBEREITEN

In [ ]:
# Falls das Archiv bereits besteht, lädt die Zelle die Tabelle 'meldungen.csv' in ein DataFrame
meldungen = pandas.read_csv('out/meldungen.csv')
meldungen.Datum = pandas.to_datetime(meldungen.Datum)

## Stellenanzeigen filtern
Filter für Stellenanzeigen in Meldungen  
Suchmaske (Regex): `'Stellenaus.*?:\s`

In [ ]:
# filtert Stellenanzeigen aus allen Meldungen 
job_pattern = 'Stellenaus.*?:\s'
stellenanzeigen = meldungen.loc[meldungen.Meldung.str.contains(job_pattern), :]
stellenanzeigen = stellenanzeigen.sort_values(by='Datum', ignore_index=True)
stellenanzeigen.Meldung.replace(to_replace=job_pattern, value='', regex=True, inplace=True)

In [ ]:
stellenanzeigen

### Stipendien / Scholarships / Stellen für Hilfskräfte löschen

In [ ]:
# löscht Stipendien / Scholarships / Stellen für Hilfskräfte löschen
stellenanzeigen.drop(stellenanzeigen.index[stellenanzeigen.Meldung.str.contains('[S|s]tip|Scholar|Hilfs')], inplace=True)
stellenanzeigen.reset_index(drop=True, inplace=True)

# III: AUSWERTUNG: STELLENANZEIGEN

### Wie viele Stellenanzeigen sind im vorliegenden Zeitraum erschienen?

In [ ]:
# Ergebnis: alle Stellenanzeigen
start = stellenanzeigen.iloc[0].Datum
end = stellenanzeigen.iloc[-10].Datum
print(f'\nIm Zeitraum von {start._date_repr} bis {end._date_repr} wurden {len(stellenanzeigen)} Stellenanzeigen gefunden.\n')

### In welchem Newsletter sind die meisten Stellenanzeigen erschienen?

In [ ]:
# In welchem Newsletter sind die meisten Stellenanzeigen erschienen?
ads_per_date = stellenanzeigen.Datum.value_counts()
ads_per_date.sort_index(inplace=True)
ads_per_date.index = pandas.to_datetime(ads_per_date.index)

print(f'\nIm Newsletter vom {str(ads_per_date.idxmax())[:10]} sind {ads_per_date.max()} Stellenanzeigen erschienen\n')

### Welche Stellenanzeigen sind an diesem Tag erschienen?

In [ ]:
stellenanzeigen.loc[stellenanzeigen.Datum == ads_per_date.idxmax()]

### Wie viele Stellenanzeigen sind pro Jahr erschienen?

In [ ]:
# Wie viele Stellenanzeigen sind pro Jahr erschienen?
startyear = 2015
endyear = 2022 # 2022 ist unvollständig, aber range-Objekt stoppt bei 2021

diagramm = {}

for year in range(startyear, endyear):
    anzahl = sum(ads_per_date[ads_per_date.index.year == year])
    diagramm[year] = anzahl

# Ausgabe als Diagramm
fig = plt.figure()
fig.set_figheight(12)
fig.set_figwidth(24)

plt.bar(diagramm.keys(), diagramm.values())
plt.title('Stellenanzeigen pro Jahr', fontsize=28)
plt.xlabel('Jahr', fontsize=20)
plt.ylabel('Stellenanzeigen', fontsize=20)
plt.style.use('ggplot')
plt.show()    

### Wie viele Stellenanzeigen sind pro Datum erschienen?

In [ ]:
# Wie viele Stellenanzeigen sind pro Datum erschienen?
# Heatmap aller Stellenausschreibungen nach Datum
import calmap
import matplotlib.pyplot as plt

c = calmap.calendarplot(ads_per_date, monthticks=3, daylabels='MDMDFSS',
                    cmap='YlOrBr',
                    fillcolor='lightgrey', linewidth=0,
                    fig_kws=dict(figsize=(40, 20)),
                    yearlabel_kws={'color':'gray', 'fontsize':48})

## AUSWERTUNG: D-A-CH-BEREICH

### Welche Stellenanzeigen sind für Standorte D-A-CH-Bereich erschienen?

In [ ]:
# ermittelt Stellenanzeigen im DACH-Bereich
# Filter für Stellenanzeigen im DACH-Bereich. Die Liste "staedte" wurde manuell ermittelt.
# (besser wäre: Suche mit Regex-Mustern)

staedte = ['Aachen','Aalen','Ansbach','Babelsberg','Basel','Bayreuth','Berlin','Bern','Bielefeld','Bochum','Bonn',\
           'Braunschweig','Bremen','Bremerhaven','Chemnitz','Chur','Cottbus','Dresden','Düsseldorf','Erfurt','Erlangen',\
           'Essen','Flensburg','Frankfurt','Freiberg','Freiburg','Friedrichshafen','Gießen','Graz','Greifswald','Göttingen',\
           'Hagen','Halle','Halle-Wittenberg','Hamburg','Hannover','Heidelberg','Hildesheim','Innsbruck','Jena','Karlsruhe',\
           'Kassel','Kiel','Klagenfurt','Konstanz','Krems','Köln','Leipzig','Linz','Ludwigsburg','Luzern','Lüneburg',\
           'Magdeburg-Stendal','Mainz','Mannheim','Marburg','München','Münster','Oldenburg','Osnabrück','Paderborn','Passau',\
           'Pforzheim','Potsdam','Regensburg','Rostock','Saarbrücken','Salzburg','Siegen','Soest','Stuttgart','Trier',\
           'Tübingen','Vechta','Weimar','Wien','Witten/Herdecke','Wuppertal','Würzburg','Zürich']


In [ ]:
# Funktion prüft, ob ein Wert aus staedte in Zeile vorhanden ist
# (Ist > 1 Wert vorhanden, wird nur der erste zurückgegeben!)
def common(staedte, row): 
    result = [stadt for stadt in staedte if re.search(stadt, row) is not None]
    if result == []:
        return None
    else:
        return result[0]

In [ ]:
#  Fügt die Spalte "Standort" hinzu
stellenanzeigen['Standort'] = stellenanzeigen['Meldung'].apply(lambda row: common(staedte, row))

In [ ]:
# filtert Standorte aus D-A-CH-Bereich
dach = stellenanzeigen.loc[stellenanzeigen.Standort.notna(), :].copy()
dach.reset_index(drop=True, inplace=True)

### Korrekturen

In [ ]:
# Korrekturen

# Halle = Halle-Wittenberg
dach.loc[dach.Standort.str.contains('Halle'), 'Standort'] = 'Halle-Wittenberg'

# Frankfurt/Oder ist nicht Frankfurt/Main
dach.loc[dach.Meldung.str.contains('Frankfurt.*Oder'), 'Standort'] = 'Frankfurt (Oder)'

In [ ]:
print(f'\nEs sind {len(dach)} Stellenanzeigen für den D-A-CH-Bereich erschienen.\n')

### Liste (DataFrame) der D-A-CH-Stellenanzeigen

In [ ]:
dach

### Welche Standorte (D-A-CH) haben die meisten Stellenanzeigen veröffentlicht?

In [ ]:
# Welche Standorte haben die meisten Stellenanzeigen veröffentlicht?
standorte_chart = dach.Standort.value_counts()
standorte_chart

### Ausgabe als Pie Chart

In [ ]:
# Ausgabe: Pie Chart (TOP 30)
labels = standorte_chart.index[:29] # nur die ersten 30
sizes = standorte_chart[:29]

fig = plt.figure()
fig.set_figheight(12)
fig.set_figwidth(24)

plt.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, radius=4, textprops={'fontsize': 30})
plt.show()

# AUSWERTUNG: STELLEN

### Definition Klasse "Stellen"

In [ ]:
class Stellen:
    
    '''
    Die Klasse wertet Stellenanzeigen nach Jobtyp aus
    Suchbegriffe müssen im searchdict stehen
    '''
    
    searchdict = {
    'Juniorprofessur':'Juniorprofess.*|W1',
    'Professur':'[pP]rofess.*',
    'Mittelbau': '[aA]kad.* [Mit.*|Rat.*]|[Ww]iss.* Mit.*|[pP]ost-?[Dd]o[ck]|Promotion.*|Doktorand|lfba|Lehrkr[aä]ft|PhD'
    }
    
    def __init__(self, suchbegriff, df=dach):
        self.suchbegriff = suchbegriff
        self.pattern = self.searchdict[self.suchbegriff]
        self.stellen = df.loc[df.Meldung.str.contains(self.pattern)]
        self.anzahl  = len(self.stellen)
        
    
    def __repr__(self):
        return self.suchbegriff
    
    def anzahl_pro_jahr(self):
        print()
        diagramm = {}
        for year in range(2015, 2022):
            anzahl = len(self.stellen.loc[self.stellen['Datum'].dt.year == year])
            diagramm[year] = anzahl

            print(f'{year}: {anzahl} Stellenanzeigen für {self.suchbegriff}')

        print()

        # Ausgabe als Diagramm
        fig = plt.figure()
        fig.set_figheight(12)
        fig.set_figwidth(24)
        
        plt.bar(diagramm.keys(), diagramm.values())
        plt.title(f'Stellenanzeigen {self.suchbegriff}', fontsize=18)
        plt.xlabel('Jahr')
        plt.ylabel(f'{self.suchbegriff}')
        plt.style.use('ggplot')
        plt.show()

### Welche Stellenanzeigen sind für den **Mittelbau** erschienen?

In [ ]:
# Filter: Stellenanzeigen für den Mittelbau
# Regex:  [aA]kad.* [Mit.*|Rat.*]|[Ww]iss.* Mit.*|[pP]ost-?[Dd]o[ck]|Promotion.*|Doktorand|lfba|Lehrkr[aä]ft
mittelbau = Stellen('Mittelbau')
mittelbau.stellen

### Wie viele Stellenanzeigen (Mittelbau) sind erschienen?

In [ ]:
print(f' \
    \nVon {len(dach)} Stellenanzeigen im D-A-CH-Bereich waren {mittelbau.anzahl} für den Mittelbau ausgeschrieben.\n\
Das sind {mittelbau.anzahl/len(dach)*100:.2f} Prozent\n')

### Verteilung der Stellenanzeigen (Mittelbau) 2015–2021

In [ ]:
mittelbau.anzahl_pro_jahr()

### Welche Stellenanzeigen für **Professuren** sind erschienen?

In [ ]:
# Filter ermittelt Stellenanzeigen für Professuren (inkl. Juniorprofessur)
professuren = Stellen('Professur')
professuren.stellen

### Wie viele Stellenanzeigen (Professur) sind erschienen?

In [ ]:
print(f'\nVon {len(dach)} Stellenanzeigen im D-A-CH-Bereich waren {professuren.anzahl} als Professuren ausgeschrieben.\n\
Das sind {professuren.anzahl/len(dach)*100:.2f} Prozent\n')

### Verteilung der Stellenanzeigen (Professur) nach Jahren

In [ ]:
professuren.anzahl_pro_jahr()

### Welche Stellenanzeigen sind für den **Juniorprofessuren** erschienen? 

In [ ]:
# Filter ermittelt Stellenanzeigen für Juniorprofessuren
# pattern = 'Juniorprofess.*|W1'
junprof = Stellen('Juniorprofessur')
junprof.stellen

### Wie viele Stellenanzeigen (Juniorprofessur) sind erschienen?

In [ ]:
anzahl1 = professuren.anzahl
anzahl2 = junprof.anzahl
print(f'\nVon {anzahl1} Stellenanzeigen für Professuren im D-A-CH-Bereich waren {anzahl2} als Juniorprofessuren/W1 ausgeschrieben.\n\
Das sind {anzahl2/anzahl1*100:.2f} Prozent\n')

### Verteilung der Stellenanzeigen (Juniorprofessur) nach Jahren

In [ ]:
# Stellenanzeigen nach Jahren
junprof.anzahl_pro_jahr()

## Inhalte von Stellenanzeigen  
Die Funktion "Auswertung" durchsucht die D-A-CH-Stellenanzeigen nach beliebig wählbaren Schlagworten  
und gibt die TOP-20-Standorte zurück

In [ ]:
# Funktion zur einfachen Auswertung von D-A-CH-Stellenanzeigen
def auswertung(pattern, df = dach):
    result = df.loc[df.Meldung.str.contains(pattern[1])]
    print(f'Es gibt {len(result)} Stellenanzeigen mit dem Stichwort \'{pattern[0]}\'\n')
    # TOP 20 der Standorte
    print(f"Die wichtigsten Standorte sind:\n\n{result.Standort.value_counts()[:20]}")

### Auswertung: Kunst / künstlerisch

In [ ]:
pattern = ('Kunst/künstlerisch', '[Kk][uü]nst.*')
auswertung(pattern)

### Auswertung: Digital

In [ ]:
pattern = ('digital', '[Dd]igital.*')
auswertung(pattern)

### Auswertung: Film

In [ ]:
pattern = ('Film', '[f|F]ilm.*')
auswertung(pattern)

### Auswertung: Fernsehen

In [ ]:
pattern = ('Fernsehen', '[f|F]ernseh.*')
auswertung(pattern)

### Auswertung: Games

In [ ]:
pattern = ('Games', 'Game.*')
auswertung(pattern)

In [ ]:
pattern = ('Fotografie', '[F|Ph]otogra.*')
auswertung(pattern)

### Vergleich von Stichworten

In [ ]:
# Funktion zur einfachen Auswertung von D-A-CH-Stellenanzeigen
result = {}

pattern_list = [
    ('Fernsehen', '[f|F]ernseh.*'),
    ('Games', 'Game.*'),
    ('Film', '[f|F]ilm.*'),
    ('digital', '[Dd]igital.*'),
    ('Kunst', '[Kk][uü]nst.*'),
    ('Fotografie', '[F|Ph]otogra.*')
]

for pattern in pattern_list:
    anzahl = len(dach.loc[dach.Meldung.str.contains(pattern[1])])
    result[pattern[0]] = anzahl

result = dict(sorted(result.items(), key = lambda x: x[1], reverse=True))  # sortiert Ergebnis nach Werten 

fig = plt.figure()
fig.set_figheight(12)
fig.set_figwidth(24)
        
plt.bar(result.keys(), result.values())
plt.title('Stichworte in Stellenanzeigen', fontsize=18)
plt.xlabel('Stichwort', fontsize=18)
plt.ylabel('Stellenanzeigen', fontsize=18)
plt.style.use('ggplot')
plt.show()

# AUSWERTUNG (II): Volltexte der STELLENANZEIGEN im Anhang

In [ ]:
df = pandas.read_csv('out/stellenanzeigen_volltext.csv')

In [ ]:
df.head(2)

### Suche im Volltext nach 'befristet'

In [ ]:
# Suche im Volltext nach 'befristet'
anzahl = len(df.loc[df.Stellenanzeige.str.contains(r'\bbefristet', regex=True), :])
print(f'Von allen {len(df)} Stellenanzeigen (im Anhang) enthalten {anzahl} den String \'befristet\'.\nDas sind {anzahl/len(df)*100:.2f} Prozent')

### Suche im Volltext nach 'unbefristet'

In [ ]:
# Suche im Volltext nach 'unbefristet'
anzahl = len(df.loc[df.Stellenanzeige.str.contains(r'unbefristet'), :])
print(f'Von allen {len(df)} Stellenanzeigen (im Anhang) enthalten {anzahl} den String \'unbefristet\'.\nDas sind {anzahl/len(df)*100:.2f} Prozent')

### Suche im Volltext nach 'Teilzeit'

In [ ]:
# Suche im Volltext nach 'Teilzeit'
anzahl = len(df.loc[df.Stellenanzeige.str.contains(r'Teilzeit'), :])
print(f'Von allen {len(df)} Stellenanzeigen (im Anhang) enthalten {anzahl} den String \'Teilzeit\'.\nDas sind {anzahl/len(df)*100:.2f} Prozent')

### Suche im Volltext nach '%'

In [ ]:
# Suche im Volltext nach 'prozentangabe (z.B. 50%, 65%, aber auch 100%)
anzahl = len(df.loc[df.Stellenanzeige.str.contains(r'\d\d\d?\s?%'), :])
print(f'Von allen {len(df)} Stellenanzeigen (im Anhang) enthalten {anzahl} eine Prozentangabe.\nDas sind {anzahl/len(df)*100:.2f} Prozent')

# Analyse der Volltexte nach Themen

In [ ]:
# analysiert ALLE Stellenanzeigen im Volltext (nicht nur D-A-CH), aber erst seit 10/2015

In [ ]:
def analyse(pattern):
    return df.Stellenanzeige.loc[df.Stellenanzeige.str.contains(pattern)].__len__()

In [ ]:
pattern_list = [
    'Cinema',
    'Film',
    'Kunst',
    'Art\s',
    'Comic',
    'Television',
    'Games',
    'Fernsehen'
]

for pattern in pattern_list:
    print(pattern, analyse(pattern))